In [2]:
import os
from glob import glob

import main_fte
from py_utils import data_ops
import evaluate_fte
from lib import app

os.environ["LOG_LOC"] = str(main_fte.__file__)
from py_utils import log

%load_ext autoreload
%autoreload 2

%matplotlib inline

# root_dir = os.path.join("..", "data")
root_dir = os.path.join("/Users/zico/OneDrive - University of Cape Town/CheetahReconstructionResults/cheetah_videos")
# If you are running the code remotely on the Linux i9.
# root_dir = os.path.join("/","data", "dlc", "to_analyse", "cheetah_videos")
logger = log.logger("main")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# Generate the pose functions if this has not been done before (note this should only be performed once). It will be saved in the root_dir.
functions = main_fte.create_pose_functions(root_dir)

In [3]:
# Configurations for the optimisation.
# 2017_08_29/top/jules/run1_1
# 2017_09_02/top/phantom/run1_3
# 2017_09_02/top/jules/run1
# 2017_08_29/top/phantom/run1_1
data_path = os.path.join("2017_09_02", "top", "jules", "run1")
start_frame = 21
end_frame = 100
cam_idx = 1
dlc_thresh = 0.5

In [5]:
# Run the optimisation
main_fte.run(root_dir, data_path, cam_idx, start_frame, end_frame, dlc_thresh, reduced_space=False, n_comps=5, include_obj_orientation=True, init_fte=False, out_dir_prefix="/Users/zico/msc/dev/AcinoSet/data")

2021-10-06T08:14:19.699 | INFO | main_fte: Prepare data - Start


Loaded extrinsics from /Users/zico/OneDrive - University of Cape Town/CheetahReconstructionResults/cheetah_videos/2017_09_02/top/extrinsic_calib/6_cam_scene_sba.json



2021-10-06T08:14:20.314 | INFO | main_fte: Load H5 2D DLC prediction data


Found 2276 pairwise points between camera 0 and 1
Found 1812 pairwise points between camera 1 and 2
Found 869 pairwise points between camera 2 and 3
Found 429 pairwise points between camera 3 and 4


2021-10-06T08:14:20.688 | INFO | main_fte: Estimate the initial trajectory


...

Ipopt 3.13.3: print_level=5
max_iter=400
max_cpu_time=10000
Tol=0.1
linear_solver=ma86
option_file_name=/var/folders/_2/61j71l3s06zfdc543275_g8w0000gn/T/tmpxdwj9ihs_ipopt.opt

Using option file "/var/folders/_2/61j71l3s06zfdc543275_g8w0000gn/T/tmpxdwj9ihs_ipopt.opt".


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.13.3, running with linear solver ma86.

Number of nonzeros in equality constraint Jacobian...:    93723
Number of nonzeros in inequality constraint Jacobian.:      160
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:    13760
                     va

2021-10-06T08:19:32.772 | INFO | main_fte: Optimisation solver took 284.55s
2021-10-06T08:19:32.776 | INFO | main_fte: Generate outputs...


Saved /Users/zico/msc/dev/AcinoSet/data/2017_09_02/top/jules/run1/fte_1/fte.pickle


2021-10-06T08:19:42.991 | INFO | main_fte: Single view error [mm] across the entire trajectory: 204.0683290922219
2021-10-06T08:19:42.995 | INFO | main_fte: Done


Saved /Users/zico/msc/dev/AcinoSet/data/2017_09_02/top/jules/run1/fte_1/cam*_fte.h5
Saved /Users/zico/msc/dev/AcinoSet/data/2017_09_02/top/jules/run1/fte_1/cam*_fte.csv



In [ ]:
dummy_scene = "/Users/zico/OneDrive - University of Cape Town/CheetahReconstructionResults/cheetah_videos/2017_08_29/top/extrinsic_calib/6_cam_scene_sba.json"
main_fte.plot_trajectory("/Users/zico/msc/dev/monocular_estimation/data/synthesized.pickle", dummy_scene)

In [ ]:
# Compare the optimisation with previous results i.e. compare the output pickle file fte.pickle. This is a visual inspection.abs
main_fte.plot_cheetah(root_dir, data_path, out_dir_prefix="/Users/zico/msc/dev/AcinoSet/data")

In [ ]:
# Compare the optimisation with previous results i.e. compare the output pickle file fte.pickle. This is a visual inspection.abs
main_fte.compare_cheetahs("/Users/zico/msc/dev/AcinoSet/data/2017_09_02/top/jules/run1/fte_1/fte.pickle", root_dir, data_path, fte_type="sd_fte", centered=False, out_dir_prefix="/Users/zico/msc/dev/AcinoSet/data")

In [ ]:
eval_dir = os.path.join("/Users/zico/msc/dev/AcinoSet/data", data_path, "fte_pw")
data = data_ops.load_pickle(os.path.join(eval_dir, "fte.pickle"))
results_dir = os.path.join(eval_dir, "evaluation")
os.makedirs(eval_dir, exist_ok=True)
max_delta_acc = evaluate_fte.eval_delta_acc(data, results_dir)
evaluate_fte.eval_model_error(data, results_dir)
evaluate_fte.eval_meas_error(data, results_dir)
logger.info(max_delta_acc)

In [ ]:
fig_fpath = os.path.join(os.path.join(os.path.join("..", "data"), data_path, "fte_pw"), 'fte.svg')
pw_file = data_ops.load_pickle(os.path.join(os.path.join("..", "data"), data_path, "fte_pw", "fte.pickle"))
app.plot_cheetah_states(pw_file["x"], out_fpath=fig_fpath)

In [ ]:
main_fte.plot_cost_functions()